In [1]:
!pip install rioxarray
!pip install geopandas
!pip install tqdm
# !pip install google.cloud
!pip install configparser
!pip install optuna
!pip install catboost
!pip install pickle5
!pip install shap
!pip install geocube
!pip install fsspec
!pip install gcsfs

     |████████████████████████████████| 46 kB 3.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 75 kB/s 
     |████████████████████████████████| 6.3 MB 59.2 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.7.1-py3-none-any.whl size=54118 sha256=c53a823d96a6e5f7fd24b4869ee95b7429e7673c38b1186bb1bd2ea0e3cd6bfb
  Stored in directory: /root/.cache/pip/wheels/42/9b/1c/acf2904f7ee44be3c0b8ffc7ea4fafebb135aee00242bef86f
Successfully built rioxarray
     |████████████████████████████████| 1.0 MB 10.7 MB/s 
     |████████████████████████████████| 15.4 MB 41 kB/s 
     |████████████████████████████████| 308 kB 12.0 MB/s 
     |████████████████████████████████| 80 kB 7.8 MB/s 
     |████████████████████████████████| 209 kB 67.7 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |██████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/03 PEMETAAN/01 MODEL - DRIVE

import os
# os.listdir()

/content/drive/MyDrive/03 PEMETAAN/01 MODEL - DRIVE


In [4]:
import pandas as pd
import optuna
import numpy as np
from catboost import Pool, EShapCalcType, EFeaturesSelectionAlgorithm,CatBoostClassifier
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import plotly.express as px
import pickle5 as pickle
from google.cloud import storage
import os
from bps_peta_processing import catb_obj,tune_param,rfe_cat,tune_study


import configparser
config = configparser.ConfigParser()
config.read('config/config_ecoregion_3001.ini')
folder=config['default']['folder']
train_no_outlier=config['output']['train_data_no_outlier']
model_param_feature=config['output']['model_param_feature']

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.


In [5]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'petabps-7c4331c6fd2d.json'

## [DON'T EDIT]

In [6]:
columns_data=['PODES_landform', 'alos_slope', 'alos_dsm', 'wet_mean',
       'green_mean', 'bright_mean', 'ARVI_mean', 'SAVI_mean', 'NDBI_mean',
       'mNDWI_mean', 'NDWI_mean', 'mNDVI_mean', 'NDVI_mean', 'wet_p50',
       'green_p50', 'bright_p50', 'ARVI_p50', 'SAVI_p50', 'NDBI_p50',
       'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50', 'S2_B12mean',
       'S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 'S2_B2mean',
       'S2_B12med', 'S2_B11med', 'S2_B8med', 'S2_B4med', 'S2_B3med',
       'S2_B2med']


## Load Data

In [7]:
file_name='kerangka-spasial/ml_learning/train_data/'+train_no_outlier
data_=pd.read_csv(file_name,sep=';').dropna()
pivot_=data_.pivot_table(index='TRAIN_CLASS',values='x',aggfunc='count').reset_index().rename(columns={'x':'SAMPLE SIZE'})
pivot_['TRAIN_CLASS']=pivot_['TRAIN_CLASS'].astype(str)
fig = px.bar(pivot_, x="TRAIN_CLASS", y="SAMPLE SIZE", title="Sample Size")
fig.show()

In [8]:
X=data_[columns_data]
y=data_[['TRAIN_CLASS']]

In [9]:
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)

In [10]:
valid_y.TRAIN_CLASS.value_counts()

11    4182
5     2994
12    1459
31    1373
9     1369
1     1364
33    1009
2      663
10     483
7      297
4      152
6       26
Name: TRAIN_CLASS, dtype: int64

## Recursive Feature Elimination

Use the RFE algorithms for feature selection with base catboost classifiers model

In [11]:
feature_select=rfe_cat(train_x,train_y,valid_x,valid_y,14)

Start Recursive Feature Elimination


Iterating Feature Elimination:   0%|          | 0/22 [00:00<?, ?it/s]

The number of features selection steps (2) is greater than the number of features to eliminate (1). The number of steps was reduced to 1.
The number of features selection steps (2) is greater than the number of features to eliminate (0). The number of steps was reduced to 0.


Best F-1 score:  0.9922581484613884
Best Number feature:  34
Selected of Feature names: 
 ['PODES_landform', 'alos_slope', 'alos_dsm', 'wet_mean', 'green_mean', 'bright_mean', 'ARVI_mean', 'SAVI_mean', 'NDBI_mean', 'mNDWI_mean', 'NDWI_mean', 'NDVI_mean', 'wet_p50', 'green_p50', 'bright_p50', 'ARVI_p50', 'SAVI_p50', 'NDBI_p50', 'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50', 'S2_B12mean', 'S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 'S2_B2mean', 'S2_B12med', 'S2_B11med', 'S2_B8med', 'S2_B4med', 'S2_B3med', 'S2_B2med']


## HyperParameter Tuning with Optuna

Tuning Parameter that would be used as model. This code take a long time

In [12]:
best_params=tune_study(train_x,train_y,valid_x,valid_y,feature_select)

[I 2021-10-13 07:01:49,538] A new study created in memory with name: no-name-40448d4c-f7d4-47e9-accd-80d34a131d3c
[I 2021-10-13 07:02:02,280] Trial 0 finished with value: 0.8641597814065447 and parameters: {'learning_rate': 0.013591611174496147, 'depth': 7, 'l2_leaf_reg': 0.0009966493462028915, 'colsample_bylevel': 0.03806047508395215, 'boosting_type': 'Plain', 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 8.648550495198306}. Best is trial 0 with value: 0.8641597814065447.
[I 2021-10-13 07:03:24,253] Trial 1 finished with value: 0.9374579836488626 and parameters: {'learning_rate': 0.043380580627361186, 'depth': 12, 'l2_leaf_reg': 0.00019687577521012645, 'colsample_bylevel': 0.09533303487365898, 'boosting_type': 'Plain', 'auto_class_weights': 'Balanced', 'bootstrap_type': 'Bernoulli', 'subsample': 0.283884548395833}. Best is trial 1 with value: 0.9374579836488626.
[I 2021-10-13 07:03:32,575] Trial 2 finished with value: 0.8584130288638779 and

Best score: 0.9638279877691757

Optimized parameters: {'learning_rate': 0.0464976478120535, 'depth': 14, 'l2_leaf_reg': 0.0009254053135587522, 'colsample_bylevel': 0.08177007704536694, 'boosting_type': 'Plain', 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'MVS'}



## Export the result


In [13]:
config_model={'features':feature_select,'params':best_params}

In [14]:
file_name = 'kerangka-spasial/ml_learning/model_result/'+model_param_feature
with open(file_name, 'wb') as handle:
    pickle.dump(config_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'petabps-7c4331c6fd2d.json'
# client = storage.Client()
# bucket = client.get_bucket('kerangka-spasial')
# file_name='kerangka-spasial/ml_learning/model_result/'+model_param_feature

# bucket.blob(file_name).upload_from_filename(model_param_feature)